In [ ]:
# 240630

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import os, json
from format import *

In [ ]:
# model_path, prompt_fn = ("meta-llama/Llama-2-70b-chat-hf", prompt_llama2)
# model_path, prompt_fn = ("upstage/SOLAR-0-70b-16bit", prompt_solar)
# model_path, prompt_fn = ("upstage/SOLAR-10.7B-Instruct-v1.0", prompt_solar)
# model_path, prompt_fn = ("moreh/MoMo-72B-lora-1.8.7-DPO", prompt_momo)
# model_path, prompt_fn = ("meta-llama/Meta-Llama-3-70B-Instruct", prompt_llama3)
# model_path, prompt_fn = ("CohereForAI/c4ai-command-r-plus", prompt_command)
# model_path, prompt_fn = ("Qwen/Qwen2-72B-Instruct", prompt_qwen2)
# model_path, prompt_fn = ("01-ai/Yi-1.5-34B-Chat", prompt_yi)
model_path, prompt_fn = ("microsoft/Phi-3-medium-4k-instruct", prompt_phi3)

### Tokenizer Test

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
messages = [
    {"role": "system", "content": "system_test"},
    {"role": "user", "content": "user_test"},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)
print("########################")
prompt = text
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
decode_text = tokenizer.decode(inputs.input_ids[0],skip_special_tokens=False)
print(decode_text)
print("########################")

In [ ]:
text = prompt_fn("system_test","user_test")
print(text)
print("########################")
prompt = text
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
decode_text = tokenizer.decode(inputs.input_ids[0],skip_special_tokens=False)
print(decode_text)
print("########################")

### Streamer Test

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    trust_remote_code=True, 
)

context= "You are a pirate chatbot who always responds in pirate speak!"
question = "안녕?"
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)
output = model.generate(
    **inputs, use_cache=True, do_sample=False, temperature=None, top_p=None, top_k=None,
    max_new_tokens=float('inf'), 
    streamer=streamer, 
    # eos_token_id=tokenizer.eos_token_id,
    # eos_token_id=[tokenizer.eos_token_id,tokenizer.convert_tokens_to_ids("<|eot_id|>")],
    # pad_token_id=tokenizer.eos_token_id,
    )
output_text = tokenizer.decode(output[0], skip_special_tokens=False)

In [ ]:
exam_path = exam_list_local[0]
with open(exam_path, encoding='utf-8') as f:
    exam = json.load(f)
    f.close()
ques_dict = exam[570]

prompt = prompt_fn(inst,ques_dict['question_text'])
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)
output = model.generate(
    **inputs, use_cache=True, do_sample=False, temperature=None, top_p=None, top_k=None,
    max_new_tokens=float('inf'), 
    streamer=streamer, 
    # eos_token_id=tokenizer.eos_token_id,
    # eos_token_id=[tokenizer.eos_token_id,tokenizer.convert_tokens_to_ids("<|eot_id|>")],
    # pad_token_id=tokenizer.eos_token_id,
    )
output_text = tokenizer.decode(output[0], skip_special_tokens=False)